### 170

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = reduce_mem_usage(df_hist)

100%|██████████| 3/3 [00:11<00:00,  3.92s/it]


Mem. usage decreased to 2554.26 Mb (0.0% reduction)


In [3]:
df = df_hist
auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

### New Merchant

In [4]:
m_id = 'merchant_id'
mc_id = 'merchant_category_id'
lag_col = 'month_lag'
lag_list = list(reversed(sorted(list(auth1[lag_col].value_counts().index))))
lag_list

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13]

In [9]:
df = auth1
df = auth0
fe_no = '_fe4'

lag = 2

# lag1_list = np.arange(6) + 1
lag1_list = (np.zeros(6)).astype('int8')
lag2_list = np.arange(6) + 3

for num, (lag1, lag2) in enumerate(zip(lag1_list, lag2_list)):
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    
    tmp = df.loc[df[lag_col].isin(lag_list[lag2:])]
    now = df.loc[df[lag_col].isin(lag_list[lag1:lag2])]
    df_now = now[[key, mc_id, m_id]].drop_duplicates()
    df_mer = tmp[[key, mc_id, m_id]].drop_duplicates()
    
    # inner joinで過去に存在したmerchantをdf_nowに残し、そのindexをdf_nowから除外することでnew_merchantのみ残す。天才！
    df_now['index'] = np.arange(len(df_now))
    mer_same = df_now.merge(df_mer, how='inner', on=[key, m_id, mc_id])
    past_mer_index = mer_same['index'].values
    df_new = df_now.loc[~df_now['index'].isin(past_mer_index), :]
    
    df_new.drop('index', axis=1, inplace=True)
    df_new.dropna(inplace=True)
    
    new_hist = df_new.merge(auth1, how='inner', on=[key, mc_id, m_id])
    print(new_hist.shape)
    print('new_hist.head()')
    display(new_hist.head())
    
    
    #========================================================================
    # aggregation feature
    #========================================================================
    df = new_hist
    fname = f'auth1_new_lag{lag1*-1}_{lag2*-1}'
    fname = f'auth0_new_lag{lag1*-1}_{lag2*-1}'
    
    train_list = []
    test_list = []
    
    def get_new_columns(name,aggs):
        return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
    
    
    def feat_agg(df, fname):
    
        aggs = {}
        
        aggs['category_1'] = ['mean', 'sum']
        
        aggs['purchase_amount'] = ['sum','max', 'min','mean']
        aggs['purchase_amount_over'] = ['sum','max', 'min','mean']
        
        aggs['merchant_id'] = ['nunique']
        aggs['merchant_category_id'] = ['nunique']
        aggs['card_id'] = ['size']
        aggs['installments'] = ['sum','max', 'min','mean', 'std']
        
        new_columns = get_new_columns(fname, aggs)
        df_agg = df.groupby(key).agg(aggs)
        df_agg.columns = new_columns
        df_agg.reset_index(drop=False,inplace=True)
         
        df_agg[f'{fname}_freq_per_90days'] = df_agg[f'{fname}_card_id_size'] / 90
        df_agg[f'{fname}_amount_per_90days'] = df_agg[f'{fname}_purchase_amount_sum'] / 90
        df_agg[f'{fname}_installments_per_90days'] = df_agg[f'{fname}_installments_sum'] / 90
        df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0e-1)
        df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0e-1)
        
        feat_train = df_train.merge(df_agg, on=key, how='left')
        feat_test = df_test.merge(df_agg, on=key, how='left')
        del df_agg
        gc.collect()
        
        print(fname)
        display(feat_train.head())
        
        train_list.append(feat_train)
        test_list.append(feat_test)
        
    feat_agg(df, fname)
    
    
    #========================================================================
    # impute
    #========================================================================
    
    def impute_agg(df):
        for col in df.columns:
            if col.count('category_1'):
                df[col].fillna(-1, inplace=True)
            if col.count('isnull'):
                df[col].fillna(-1, inplace=True)
            if col.count('null_cnt'):
                df[col].fillna(-1, inplace=True)
            if col.count('month_lag'):
                df[col].fillna(df[col].min()-15, inplace=True)
            if col.count('month_diff'):
                df[col].fillna(df[col].max()+100, inplace=True)
            if col.count('nunique'):
                df[col].fillna(-1, inplace=True)
            if col.count('amount'):
                df[col].fillna(-1, inplace=True)
            if col.count('installments'):
                df[col].fillna(-2, inplace=True)
            if col.count('unix_date'):
                df[col].fillna(df[col].min()-100000, inplace=True)
            if col.count('size'):
                df[col].fillna(-1, inplace=True)
            if col.count('term'):
                df[col].fillna(-1, inplace=True)
            if col.count('per'):
                df[col].fillna(-1, inplace=True)
            if col.count('dummie'):
                df[col].fillna(-1, inplace=True)
                
        return df
            
    for train, test in zip(train_list, test_list):
        try:
            train.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
            test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month'], axis=1, inplace=True)
        except KeyError:
            pass
        
        try:
            train.set_index(key, inplace=True)
            test.set_index(key, inplace=True)
        except KeyError:
            pass
        
        df_train = df_train.join(train)
        df_test = df_test.join(test)
    
    df_train = impute_agg(df_train)
    df_test = impute_agg(df_test)
    
    display( df_train.head())
    
    
    #========================================================================
    # Save Feature
    #========================================================================
    fname = f'{172+num}' + fe_no
    ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
    
    for col in df_train.columns:
        if col in ignore_features: continue
        if (col.count('feature_')):continue
        if (col.count('purchase_date')):continue
        feat_train = df_train[col].astype('float32').values
        try:
            feat_test = df_test[col].astype('float32').values
        except TypeError:
            print(col)
            sys.exit()
        col = col.replace('.', '_')
        utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=feat_train)
        utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=feat_test)
#         utils.to_pkl_gzip(path = f'../season1_features/1_first_valid/{fname}_train_{col}@', obj=feat_train)
#         utils.to_pkl_gzip(path = f'../season1_features/1_first_valid/{fname}_test_{col}@', obj=feat_test)

100%|██████████| 3/3 [00:00<00:00, 179.95it/s]


(1553365, 21)
new_hist.head()


,card_id,merchant_category_id,merchant_id,purchase_date,authorized_flag,city_id,category_1,installments,category_3,month_lag,purchase_amount,category_2,state_id,subsector_id,first_active_month,all_term,yyyymm,yyyymmdd,weekofyear,yyyy_week,purchase_amount_over
0,C_ID_4e6213e9bc,605,M_ID_c2ae34c2ef,2018-02-20 10:58:17,1,333,0,0.0,A,0,0.082642,1.0,9,2,2016-06-01,699,2018-02,2018-02-20,8,2018-08,0.0
1,C_ID_0e171c1b48,705,M_ID_39fd61764d,2017-11-01 19:32:20,1,277,0,0.0,A,-3,0.229858,4.0,13,33,2017-03-01,426,2017-11,2017-11-01,44,2017-44,0.0
2,C_ID_0e171c1b48,705,M_ID_39fd61764d,2017-12-23 18:10:59,1,277,0,0.0,A,-2,0.030060,4.0,13,33,2017-03-01,426,2017-12,2017-12-23,51,2017-51,0.0
3,C_ID_0e171c1b48,705,M_ID_39fd61764d,2017-10-05 00:15:08,1,277,0,0.0,A,-4,0.030869,4.0,13,33,2017-03-01,426,2017-10,2017-10-05,40,2017-40,0.0
4,C_ID_0e171c1b48,307,M_ID_4d34f517e7,2018-01-06 14:57:19,1,277,0,0.0,A,-1,0.015793,4.0,13,19,2017-03-01,426,2018-01,2018-01-06,1,2018-01,0.0


auth0_new_lag0_-3


,card_id,first_active_month,feature_1,feature_2,feature_3,target,auth0_new_lag0_-3_category_1_mean,auth0_new_lag0_-3_category_1_sum,auth0_new_lag0_-3_purchase_amount_sum,auth0_new_lag0_-3_purchase_amount_max,auth0_new_lag0_-3_purchase_amount_min,auth0_new_lag0_-3_purchase_amount_mean,auth0_new_lag0_-3_purchase_amount_over_sum,auth0_new_lag0_-3_purchase_amount_over_max,auth0_new_lag0_-3_purchase_amount_over_min,auth0_new_lag0_-3_purchase_amount_over_mean,auth0_new_lag0_-3_merchant_id_nunique,auth0_new_lag0_-3_merchant_category_id_nunique,auth0_new_lag0_-3_card_id_size,auth0_new_lag0_-3_installments_sum,auth0_new_lag0_-3_installments_max,auth0_new_lag0_-3_installments_min,auth0_new_lag0_-3_installments_mean,auth0_new_lag0_-3_installments_std,auth0_new_lag0_-3_freq_per_90days,auth0_new_lag0_-3_amount_per_90days,auth0_new_lag0_-3_installments_per_90days,auth0_new_lag0_-3_amount_per_installments_sum,auth0_new_lag0_-3_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,0.0,0.0,0.698242,0.195312,0.018036,0.069824,0.000000,0.000000,0.000000,0.000000,2.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.007759,0.000000,6.984375,0.698242
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,0.0,0.0,0.524414,0.077393,0.006771,0.020966,0.000000,0.000000,0.000000,0.000000,1.0,1.0,25.0,25.0,1.0,1.0,1.0,0.0,0.277778,0.005825,0.277832,0.020905,0.019073
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,1.0,1.0,2.000000,2.000000,2.000000,2.000000,7.738281,7.738281,7.738281,7.738281,1.0,1.0,1.0,12.0,12.0,12.0,12.0,NaN,0.011111,0.022217,0.133301,0.165283,0.165283


...

auth0_new_lag0_-5


,card_id,first_active_month,feature_1,feature_2,feature_3,target,auth0_new_lag0_-5_category_1_mean,auth0_new_lag0_-5_category_1_sum,auth0_new_lag0_-5_purchase_amount_sum,auth0_new_lag0_-5_purchase_amount_max,auth0_new_lag0_-5_purchase_amount_min,auth0_new_lag0_-5_purchase_amount_mean,auth0_new_lag0_-5_purchase_amount_over_sum,auth0_new_lag0_-5_purchase_amount_over_max,auth0_new_lag0_-5_purchase_amount_over_min,auth0_new_lag0_-5_purchase_amount_over_mean,auth0_new_lag0_-5_merchant_id_nunique,auth0_new_lag0_-5_merchant_category_id_nunique,auth0_new_lag0_-5_card_id_size,auth0_new_lag0_-5_installments_sum,auth0_new_lag0_-5_installments_max,auth0_new_lag0_-5_installments_min,auth0_new_lag0_-5_installments_mean,auth0_new_lag0_-5_installments_std,auth0_new_lag0_-5_freq_per_90days,auth0_new_lag0_-5_amount_per_90days,auth0_new_lag0_-5_installments_per_90days,auth0_new_lag0_-5_amount_per_installments_sum,auth0_new_lag0_-5_amount_per_installments_mean
0,C_ID_92a2005557,2017-06,5,2,1,-0.820283,0.0,0.0,0.018036,0.018036,0.018036,0.018036,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,0.011111,0.000200,0.000000,0.180420,0.180420
1,C_ID_3d0044924f,2017-01,4,1,0,0.392913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C_ID_d639edf6cd,2016-08,2,2,0,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,1.0,1.0,2.000000,2.000000,2.000000,2.000000,7.738281,7.738281,7.738281,7.738281,1.0,1.0,1.0,12.0,12.0,12.0,12.0,NaN,0.011111,0.022217,0.133301,0.165283,0.165283


,first_active_month,feature_1,feature_2,feature_3,target,auth0_new_lag0_-5_category_1_mean,auth0_new_lag0_-5_category_1_sum,auth0_new_lag0_-5_purchase_amount_sum,auth0_new_lag0_-5_purchase_amount_max,auth0_new_lag0_-5_purchase_amount_min,auth0_new_lag0_-5_purchase_amount_mean,auth0_new_lag0_-5_purchase_amount_over_sum,auth0_new_lag0_-5_purchase_amount_over_max,auth0_new_lag0_-5_purchase_amount_over_min,auth0_new_lag0_-5_purchase_amount_over_mean,auth0_new_lag0_-5_merchant_id_nunique,auth0_new_lag0_-5_merchant_category_id_nunique,auth0_new_lag0_-5_card_id_size,auth0_new_lag0_-5_installments_sum,auth0_new_lag0_-5_installments_max,auth0_new_lag0_-5_installments_min,auth0_new_lag0_-5_installments_mean,auth0_new_lag0_-5_installments_std,auth0_new_lag0_-5_freq_per_90days,auth0_new_lag0_-5_amount_per_90days,auth0_new_lag0_-5_installments_per_90days,auth0_new_lag0_-5_amount_per_installments_sum,auth0_new_lag0_-5_amount_per_installments_mean
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,0.0,0.0,0.018036,0.018036,0.018036,0.018036,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,-2.0,0.011111,0.000200,0.000000,0.180420,0.180420
C_ID_3d0044924f,2017-01,4,1,0,0.392913,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.000000,-1.000000,-2.000000,-1.000000,-1.000000
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.000000,-1.000000,-2.000000,-1.000000,-1.000000
C_ID_186d6a6901,2017-09,4,3,0,0.142495,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.000000,-1.000000,-2.000000,-1.000000,-1.000000
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,1.0,1.0,2.000000,2.000000,2.000000,2.000000,7.738281,7.738281,7.738281,7.738281,1.0,1.0,1.0,12.0,12.0,12.0,12.0,-2.0,0.011111,0.022217,0.133301,0.165283,0.165283


100%|██████████| 3/3 [00:00<00:00, 163.86it/s]


KeyboardInterrupt: 